# Simulate canonical olfaction as in Qin et al. 2019

Here we are following exactly the notation from their paper

In [37]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import differential_entropy
from scipy.stats import norm, rankdata
from scipy.special import psi

In [38]:
class OlfactorySensing():
    def __init__(self, N=100, n=2, M = 30, sigma_0=1e-2, sigma_c=2): 
        self.N = N
        self.n = n
        self.M = M
        self.sigma_0 = sigma_0
        self.sigma_c = sigma_c
        self.set_sigma() 
        self.set_random_W()

    def draw_c(self): 
        c = np.zeros(self.N)
        non_zero_indices = np.random.choice(self.N, self.n, replace=False)
        # Generate log-normal concentrations for these chosen odorants
        concentrations = np.random.lognormal(mean=0, sigma=self.sigma_c, size=self.n)
        c[non_zero_indices] = concentrations
        # we should think about how to do this better. If you put correlation structure in, it should affect both concentration and presence. Otherwise because n << N your mixtures will in practice be correlated. 
        return c
    
    def draw_cs(self, P):
        self.c = np.zeros((self.N, P))
        for p in range(P): 
            self.c[:, p] = self.draw_c()

    def set_sigma(self): 
        self.sigma = lambda x: x / (1 + x) 

    def set_random_W(self): 
        self.W = np.random.normal(loc=0, scale=1, size=(self.M, self.N))

    def compute_activity(self): 
        self.r = self.sigma(self.W @ self.c) + np.random.normal(loc=0, scale=self.sigma_0)

    def compute_entropy_of_r(self):
        entropy = self.compute_marginal_entropies() - self.compute_information_of_r()
        self.entropy = entropy 

    def compute_marginal_entropies(self):
        marginal_entropies = []
        for m in range(self.M): 
            marginal_entropies.append(differential_entropy(self.r[m, :]))
        return np.sum(marginal_entropies) 
    
    def compute_information_of_r(self): 
        # as in Qin et al. 2019, we use gaussian copula. This is nothing more than transforming your data through inverse normal cdf to have marginal gaussian distributions. 
        M, P = self.r.shape  # [dimension, sample size]
        # Step 1: Transform data to approximate standard normal in each dimension
        G = norm.ppf((rankdata(self.r.T, axis=0) / (P + 1)), loc=0, scale=1)
        bias_correction = 0.5 * np.sum(psi((P - np.arange(1, M + 1) + 1) / 2) - np.log(P / 2))
        # Step 3: Log determinant using Cholesky decomposition
        cov_matrix = np.cov(G, rowvar=False)
        chol_decomp = np.linalg.cholesky(cov_matrix)
        log_det = np.sum(np.log(np.diag(chol_decomp)))
        # Step 4: Mutual information estimate
        I = -(log_det - bias_correction)
        return I

In [39]:
os = OlfactorySensing()


In [40]:
os.draw_cs(P=1000)

In [41]:
os.compute_activity()

In [42]:
os.r.shape

(30, 1000)

In [43]:
os.compute_information_of_r()

3.5651622243442183

In [45]:
os.compute_entropy_of_r()